In [2]:
Tensor = list

In [3]:
from typing import List


def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0] if tensor else []
    return sizes


assert shape([1, 2, 3]) == [3]
assert shape([[1, 2], [3, 4], [5, 6]]) == [3, 2]
assert shape([
    [[1, 2, 3, 4],
     [5, 6, 7, 8],
     [9, 10, 11, 12]],
    [[13, 14, 15, 16],
     [17, 18, 19, 20],
     [21, 22, 23, 24]]
]) == [2, 3, 4]

In [4]:
def is_1d(tensor: Tensor) -> bool: 
    """ 
    If tensor[0] is a list, it's a higher-order tensor. 
    Otherwise, tensor is 1-dimensional (that is, a vector). 
    """ 
    return not isinstance(tensor[0], list) 
 
assert is_1d([1, 2, 3]) 
assert not is_1d([[1, 2], [3, 4]])

In [5]:
def tensor_sum(tensor: Tensor) -> float: 
    """Sums up all the values in the tensor""" 
    if is_1d(tensor): 
        return sum(tensor)  # just a list of floats, use Python sum 
    else: 
        return sum(tensor_sum(tensor_i)      # Call tensor_sum on each row 
                   for tensor_i in tensor)   # and sum up those results. 
 
assert tensor_sum([1, 2, 3]) == 6 
assert tensor_sum([[1, 2], [3, 4]]) == 10

In [6]:
from typing import Callable

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    """Applies a function to each element in the tensor."""
    if is_1d(tensor):
        return [f(x) for x in tensor]
    else:
        return [tensor_apply(f, sub_tensor) for sub_tensor in tensor]

assert tensor_apply(lambda x: x + 1, [1, 2, 3]) == [2, 3, 4] 
assert tensor_apply(lambda x: 2 * x, [[1, 2], [3, 4]]) == [[2, 4], [6, 8]]

In [8]:
import operator

def tensor_combine(f: Callable[[float, float], float], 
                   tensor1: Tensor, tensor2: Tensor) -> Tensor:
    """Combines two tensors element-wise using a binary function."""
    if is_1d(tensor1) and is_1d(tensor2):
        return [f(x, y) for x, y in zip(tensor1, tensor2)]
    else:
        return [tensor_combine(f, sub_tensor1, sub_tensor2)
                for sub_tensor1, sub_tensor2 in zip(tensor1, tensor2)]

assert tensor_combine(operator.add, [1, 2, 3], [4, 5, 6]) == [5, 7, 9]
assert tensor_combine(operator.mul, [1, 2, 3], [4, 5, 6]) == [4, 10, 18]
assert tensor_combine(operator.mul, [[1, 2], [3, 4]], [[5, 6], [7, 8]]) == [[5, 12], [21, 32]]

In [ ]:
from typing import Iterable, Tuple

class Layer:
    """ 
    Our neural networks will be composed of Layers, each of which 
    knows how to do some computation on its inputs in the "forward" 
    direction and propagate gradients in the "backward" direction. 
    """
    def forward(self, input): 
        """ 
        Note the lack of types. We're not going to be prescriptive 
        about what kinds of inputs layers can take and what kinds 
        of outputs they can return. 
        """ 
        raise NotImplementedError

    def backward(self, gradient): 
        """ 
        Similarly, we're not going to be prescriptive about what the 
        gradient looks like. It's up to you the user to make sure 
        that you're doing things sensibly. 
        """ 
        raise NotImplementedError

    def params(self) -> Iterable[Tensor]: 
        """ 
        Returns the parameters of this layer. The default implementation 
        returns nothing, so that if you have a layer with no parameters 
        you don't have to implement this. 
        """ 
        return ()
    
    def grads(self) -> Iterable[Tensor]: 
        """ 
        Returns the gradients, in the same order as params(). 
        """ 
        return ()

In [ ]:
from scratch.neural_networks import sigmoid

class Sigmoid(Layer):
    def forward(self, input: Tensor) -> Tensor:
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids        
    
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(
            lambda sig, grad: sig * (1 - sig) * grad,
            self.sigmoids, 
            gradient)